<a href="https://colab.research.google.com/github/IkuStudies/abraxalexicon/blob/main/Colab2Milvus2Abraxa(v0_1BETA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abraxa Lexicon Search guide

1. Run the first cell.  install dependencies.  wait for green checkmark.

2. now go to menu above, click "Runtime" select "restart runtime". IMPORTANT (You must restart the runtime or you will get conflicting dependency errors, it needs to register the new packages with a restart)

3. you don't need to run the first cell again.  scroll down to second cell.  click play.  it will ask you for a word at the bottom of the cell.  

4. type your word in, press enter.  results will be printed in the output.  the buttons are not functional yet this is all in BETA, working on getting an IPA reader to read the IPA (international phonetic alphabet)

5. to search for hebrew similar words remove all vowels

any questions or interested in developing some related tools, join ikustudygroup, https://t.me/ikustudygroup

Josh T
IkuStudies

PS.  I just saw that there's something i overlooked in the embeddings, there's some diacritic marks that are mbedded as zero which they should just be ignored for this, so I need to update the entire database to take out all the diacritics then re-embed it.

its about a 4 hour job so once i get around to that the results will be wayyy better.


In [7]:
%%capture
!pip install pymilvus deep_translator pyttsx3 ipywidgets eng_to_ipa
!apt-get install espeak 


In [12]:
import pyttsx3
import ipywidgets as widgets
from IPython.display import display
from pymilvus import connections, Collection
import json
from deep_translator import GoogleTranslator
import eng_to_ipa as ipa
import re

# IPA mapping
ipa_mapping = {
    'ʉ': 1, 'ɯ': 1, 'u': 1, 'ʏ': 1, 'ʊ': 1,
    'ɵ': 2, 'ɤ': 2, 'o': 2, 'ə': 2, 'ɤ̞': 2,
    'e̞': 3, 'ø̞': 3, 'ɛ': 3, 'œ': 3, 'ɶ': 3, 'ɜ': 3, 'ɞ': 3, 'e': 3, 'ø': 3, 'ɘ': 3, 'ɪ': 3, 'i': 3, 'y': 3, 'ɨ': 3,
    'ɐ': 4, 'ʌ': 4, 'ʡ': 4, 'ʔ': 4, 'ʔ̞': 4, 'ʡ̆': 4,
    'æ': 5, 'a': 5, 'ä': 5,
    'o̞': 6, 'ɔ': 6, 'ɑ': 6, 'ɒ': 6,
    'm̥': 7, 'm': 7, 'ɱ': 7, 'ɳ̊': 7,
    'n̼': 8, 'n̥': 8, 'n': 8, 'ɳ': 8, 'ɲ̊': 8, 'ɲ': 8, 'ŋ̊': 8, 'ŋ': 8, 'ɴ': 8, 'n': 8,
    'p': 9, 'b': 9, 'p̪': 9, 'b̪': 9, 'ʙ̥': 9, 'ʙ': 9, 'ɹ̥': 9,
    'd̼': 10, 'd': 10, 'ɖ': 10,
    't': 11, 'ʈ': 11, 't̼': 11,
    'j': 12, 'ɟ': 12,
    'k': 13, 'q': 13, 'x': 13, 'χ': 13, 'ħ': 13, 'c': 13,
    'ɣ': 14, 'ɡ': 14, 'ɢ': 14, 'ɢ̆': 14, 'g':14,
    'z': 15, 'ɮ': 15, 'ʃ': 15, 'ʒ': 15, 'ʂ': 15, 'ʐ': 15, 'ɕ': 15, 'ʑ': 15, 'ʝ': 15, 'ɕ': 15, 'ʑ': 15, 's': 15,
    'ɸ': 16, 'β': 16, 'f': 16, 'v': 16, 'ⱱ̟': 16, 'ⱱ': 16,
    'θ̼': 17, 'ð̼': 17, 'θ': 17, 'ð': 17, 'θ̠': 17, 'ð̠': 17, 'ɹ̠̊˔': 17, 'ɹ̠˔': 17, 'ɻ̊˔': 17, 'ɻ˔': 17, 'th': 17,
    'ɺ̥': 18, 'ɺ': 18, 'ɾ̼': 18, 'ɾ̥': 18, 'ɾ': 18, 'ɽ̊': 18, 'ɽ': 18, 'ɹ': 18, 'ɻ': 18, 'ʁ': 18,
    'ç': 19, 'ʕ': 19, 'h': 19, 'ɦ': 19,
    'ɰ': 20, 'w': 20,
    'r': 21, 'r̥': 21, 'ɽ̊r̥': 21, 'ɽr': 21, 'ʀ̥': 21, 'ʀ': 21,
    'ʜ': 22, 'ʢ': 22,
    'ɬ': 23, 'ꞎ': 23, 'ʎ̝': 23, 'ʟ̝': 23, 'l': 23, 'ɭ': 23, 'ʎ': 23, 'ʟ': 23, 'ʟ̠': 23, 'ʎ̆': 23, 'ʟ̆': 23
}


def word_to_embedding(word):
    embedding = []
    for char in word:
        if char in ipa_mapping:
            embedding.append(ipa_mapping[char])
        else:
            embedding.append(0)
    while len(embedding) < 18:
        embedding.append(0)
    return embedding

def speak_word(word):
    engine.say(word)
    engine.runAndWait()

def button_click(event):
    word = event.description
    speak_word(word)

def convert_to_ipa(word):
    ipa_word = ipa.convert(word)
    # Remove diacritics from the IPA word
    ipa_word = re.sub(r"ˈ|ˌ|̩|̯", "", ipa_word)
    return ipa_word

# Initialize the speech synthesis engine
engine = pyttsx3.init()

# Milvus connection
alias = "default"
milvus_uri = "https://in01-4d495aa932f01f7.aws-us-west-2.vectordb.zillizcloud.com:19538"  # Replace with your Milvus URI
user = "SpaceDuck420"  # Replace with your Milvus username
password = "Spaceduck69!"  # Replace with your Milvus password

connections.connect(alias, uri=milvus_uri, user=user, password=password, secure=True)
print(f"Connecting to Milvus: {milvus_uri}")


# Get user input word
input_word = input("Enter a word:")
ipa_word = convert_to_ipa(input_word)
print(f"Word: {input_word}")
print(f"IPA: {ipa_word}")
embedding_array = word_to_embedding(ipa_word)
print(f"Embedding: {embedding_array}")

# Search Milvus
collection = Collection("lexicon")
collection.load()

search_params = {
    "metric_type": "L2" #a few possible options to try here: MetricType.JACCARD MetricType.COSINE MetricType.EUCLIDIAN?
}

results = collection.search(
    data=[embedding_array],
    anns_field="embedding",
    param=search_params,
    limit=100,
    #expr=None, #i don't know wtf this is but i'm adding it to experiment cuz i saw it in someone elses search.  comment out at your own risk.  if you mess it up just disconnect and start over. 
    #ah ok here https://milvus.io/docs/boolean.md 
    #partition_names ??? can this relate to id codes?  this might be the avenue for language choice
    output_fields=['id', 'word', 'pronunciation'],
    #consistency_level="Eventually" #uncomment this line out to experiment.  options are Strong, Bounded, Session, or Eventually.  if uncomment add comma to end of line above.
)

# Translation
translated_input_word = GoogleTranslator(source='auto', target='en').translate(input_word)
langs_dict = GoogleTranslator().get_supported_languages(as_dict=True)
print(f"Translated: {translated_input_word}")

# Display search results
for hit in results[0]:
    word = hit.entity.get('word')
    pronunciation = hit.entity.get('pronunciation')
    translated_word = GoogleTranslator(source='auto', target='en').translate(word)
    formatted_output = f"id: {hit.id}, word: {word}, pronunciation: {pronunciation}, translation: {translated_word}"
    print(formatted_output)

# Create buttons for each word
buttons = []
for hit in results[0]:
    word = hit.entity.get('word')
    translation = GoogleTranslator(source='auto', target='en').translate(hit.entity.get('word'))
    button_description = f"{word} | {translation}"
    button = widgets.Button(description=button_description)
    button.on_click(button_click)
    buttons.append(button)

# Display buttons
display(*buttons)

connections.disconnect(alias)




Connecting to Milvus: https://in01-4d495aa932f01f7.aws-us-west-2.vectordb.zillizcloud.com:19538
Enter a word:cheetoh
Word: cheetoh
IPA: cheetoh*
Embedding: [13, 19, 3, 3, 11, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Translated: cheetoh
id: ar_2645163, word: كهيئتيه, pronunciation: khiːiːٔtiːh, translation: What is it?
id: ar_2645149, word: كهويتيه, pronunciation: khuːiːtiːh, translation: his identity
id: ar_2610645, word: كعائديه, pronunciation: kʕaːiːٔdiːh, translation: As his return
id: ar_2613586, word: كعياديه, pronunciation: kʕiːaːdiːh, translation: As a clinic
id: en_327493, word: traiteur, pronunciation: t_ɹ_eɪ_t_ˈʊɹ, translation: caterer
id: tk_34988, word: kreatör, pronunciation: kɾeatœɾ, translation: Creator
id: hi_150195, word: क्रिएटर, pronunciation: krieʈər, translation: creator
id: ar_2598033, word: كرايتيه, pronunciation: kraːiːtiːh, translation: Krate
id: ar_2645174, word: كهيابيه, pronunciation: khiːaːbiːh, translation: khiabih
id: ar_2600213, word: كرويتيه, pronunciati

Button(description='كهيئتيه | What is it?', style=ButtonStyle())

Button(description='كهويتيه | his identity', style=ButtonStyle())

Button(description='كعائديه | As his return', style=ButtonStyle())

Button(description='كعياديه | As a clinic', style=ButtonStyle())

...

Button(description='tweeterais | would tweet', style=ButtonStyle())

Button(description='tweeterait | would tweet', style=ButtonStyle())

Button(description='tweeterez | will tweet', style=ButtonStyle())

Button(description='فعوادوه | So they returned him', style=ButtonStyle())

Button(description='فرويديه | Freudian', style=ButtonStyle())

#

#

#

#

#

#

#

#

#dev notes

#

#

#

#

#

#

#

#



a few notes.  this is fully beta.  i'm still hashing out how tf milvus works so we can configure it.  here's some documentation https://milvus.io/docs/search.md

also https://milvus.io/docs/consistency.md

the results are fun now, but i am basically doing an ANN search, which stands for Nearest Neighbor, thats why theres so many wildcards in the mix i think.  I'd like to have all exact matches first in the results.  

I know there must be a work around to get anagrams as well.  its just alot to wrap my mind around, i'm surprised i finally got it to actually work this much. 

now the embedding scheme i came up with for all the IPA pronunciation fields can be seen above in the code.  that can be improved for sure.

also note to self, I should really change the input word embedding to go to epitran to phoneme to embedding rather than straight to embedding, because letters like C act as both K and S so epitran will sort that out and the results will be tighter.

i also have like 20 more wordlists with probably another 8 million words ready to insert into the vector memory, but i would like to fine tune the search parameters.

also i think i'm running into the same problem as this girl ran into in this article https://cuttlesoft.com/blog/2018/09/13/pronouncing-things-with-amazons-polly/ i'm not quite sure how to do anything but program in python just cuz its the game i play.  so i'm not sure how to implement her solution for my buttons but i would like to have buttons that play the pronunciations for reference.  especially with these foreign scripts.  but i disabled the buttons since it was just reading the names of the letters rather than the sound the letters make.  

i can see about 10 things that will improve this.  I think also setting a configuration scheme into the code is a good idea

to have like 

English=True

Hebrew=True

Greek=True

Arabic=False

Farsi=False

etc etc

for which partitions to include in any search.

also it would be massively beneficial to host the translations natively and get better context for future ML models to detect cognates by being cogniscent of both translation and pronunciation(translation is an after thought since getting 10 million+ translation requires alot of API calls and takes forever, it would have taken me 2 weeks just to do arabic so i just changed it to grab translations after results were returned...

of course aryan languages are packed with cognates, but i'm more interested in new world cognates with old world languages.  especially navajo and chinese, and quechua and egyptian.

also i'd like to do a full on japanese to hebrew search

japanese is broken in this database, so many of the words didn't get translated so i need to work on fixing that as well

its a bit buggy.  if anybody is pretty good with python etc and wants to get their hands dirty and help improve the project it'll be pretty novel lmk https://t.me/ikustudygroup is sort of where i'm keeping notes 

i think the project is worth a research grant and hiring some help.

there's a pretty cool vision from this stepping stone ... next step is V2 Quipu which is native american languages in complete corpus, and then V3 is pyramidion which is the capstone, and that's fully training a machine learning model on hieroglyphs and succeeding where others have given up.  

and getting the full corpus of ancient egypt into the database

so this is beta of V1 Abraxa... V2 Quipu... V3 Pyramidion

and then release it as an open source research project 

i bought https://pyramidion.org today and asked my mom to register a nonprofit to apply for research grants through.  i think its feasable.  and if you know python modules, PYramidion is a perfect python module name.  it will be like pytesseract on steroids and crack and other drugs

https://en.wikipedia.org/wiki/Abraxas (abraxa is like girl version of abraxas)
https://en.wikipedia.org/wiki/Pyramidion
https://en.wikipedia.org/wiki/Quipu 

so thats the game plan.  

if anybody wants to get fully immersed i have some relatively easy tasks i need help with.  like for example translating all that japanese.  and soon chinese.  

also i have a series of code blocks that i just need to basically reprocess a bunch of json files with the new embeddings, its completely easy.  just time consuming.

but whats going to be difficult is figuring out the new embedding scheme to be most conducive to really capturing the linguistic data across corpuses.  

that's where its at.  i just discovered some milvus documentation i'm hoping to find some way to really fine tune this.  i think the key will be in 

"Milvus supports setting consistency level specifically for a search. The example in this topic sets the consistency level as Strong. You can also set the consistency level as Bounded, Session or Eventually. See Consistency for more information about the four consistency levels in Milvus."

something about this to tighten or loosen...

will try to figure it out, i need to join some kind of milvus community discord or something, there seems to be few resources on milvus that i can find.  

def looking for professional help if anybody's got some freetime and is on the level.  I'm licensing this whole thing as opensource, also gotta figure out a sustainable hosting solution for the database since it costs 50 cents an hour to run on cloud.

#join noteskeeping and occasional discussion, might go live with this thing soon at 

#https://t.me/ikustudygroup 

all technical wizards are super invited to collaborate and fine tune, optimize, and find solutions to make it a more useful linguistics study tool.  once you see how its engineered i think its pretty straight forward.  alright enjoy your homophone hunting. later
Josh
@Ikustudies

In [ ]:
a few quick notes


To change the similarity metrics in Python from Euclidean distance to Jaccard similarity to Cosine similarity, 
you can use the search function provided by the Milvus Python SDK. The search function takes the following parameters:

    collection_name: The name of the collection to search in.
    query_entities: A list of query entities to search for.
    top_k: The number of nearest neighbors to return.
    params: A dictionary of search parameters, including the similarity metric.

To change the similarity metric, you can set the metric_type parameter in the params dictionary to the desired similarity metric. 
For example, to use Jaccard similarity, you can set metric_type to MetricType.JACCARD. Similarly, to use Cosine similarity, you can set metric_type to MetricType.COSINE.

Here's an example code snippet that demonstrates how to change the similarity metric:

from milvus import Milvus, IndexType, MetricType

# Connect to Milvus server
milvus = Milvus(host='localhost', port='19530')

# Set the collection name
collection_name = 'my_collection'

# Set the query entities
query_entities = [[0.1, 0.2, 0.3, 0.4]]

# Set the number of nearest neighbors to return
top_k = 10

# Set the search parameters
params = {'metric_type': MetricType.JACCARD}

# Search for nearest neighbors
results = milvus.search(collection_name=collection_name, query_entities=query_entities, top_k=top_k, params=params)

# Print the search results
print(results)

In this example, the params dictionary is set to use Jaccard similarity as the similarity metric. You can modify the params dictionary to use other similarity metrics as well.



To change the index type of a collection in Milvus using Python, you can use the modify_collection f
unction from the pymilvus library. The modify_collection function allows you to modify the properties of a collection, including the index type.

To change the index type, you need to pass the index_type parameter to the modify_collection function. 
The available index types in Milvus include FLAT, IVF_FLAT, IVF_SQ8, RNSG, IVF_PQ, and HNSW.

Here's an example code snippet that shows how to change the index type of a collection to IVF_FLAT:

from pymilvus import Collection, connections

connections.connect()

collection = Collection('my_collection')
collection.modify_collection(index_type='IVF_FLAT')

connections.disconnect()

You can replace my_collection with the name of your collection.

Source: Milvus documentation



Milvus supports several index types, each with its own use cases and functionalities. Here's a brief overview of the different index types in Milvus:

  FLAT: This is a simple index type that stores vectors in a flat table. It is best suited for small datasets with low dimensionality.

  IVF_FLAT: This index type is based on the inverted file structure and is designed for large-scale datasets. It divides the dataset into multiple subspaces and stores the vectors in each subspace in a separate inverted file. It is best suited for datasets with high dimensionality.

  IVF_SQ8: This index type is similar to IVF_FLAT, but it uses a quantization algorithm to compress the vectors before storing them in the inverted file. It is best suited for datasets with high dimensionality and large cardinality.

  RNSG: This index type is based on the random projection algorithm and is designed for datasets with high dimensionality. It builds a graph structure to represent the dataset and uses a random walk algorithm to search for similar vectors.

  IVF_PQ: This index type is similar to IVF_FLAT, but it uses a product quantization algorithm to compress the vectors before storing them in the inverted file. It is best suited for datasets with high dimensionality and large cardinality.

  HNSW: This index type is based on the hierarchical navigable small world graph algorithm and is designed for datasets with high dimensionality. It builds a graph structure to represent the dataset and uses a heuristic search algorithm to search for similar vectors.

Each index type has its own strengths and weaknesses, and the choice of index type depends on the characteristics of the dataset and the requirements of the application. For example, HNSW is best suited for scenarios that have a high demand for search efficiency, while ANNOY is best suited for scenarios that seek a high recall rate.

just found this extremely helpful milvus bot https://osschat.io/chat?project=Milvus

In [ ]:
#to see what index type is set for future reference

from pymilvus import Collection

collection = Collection("<collection_name>")
index_type = collection.index_type

print(index_type)